In [164]:
import requests  # 导入必要的库
import matplotlib.pyplot as plt  # 导入必要的库
import pandas as pd  # 导入必要的库
from lxml import etree  # 导入必要的库
from selenium import webdriver  # 导入必要的库

In [165]:
import time  # 导入必要的库
import random  # 导入必要的库

## 评论提取

爬取网址示范：https://guba.eastmoney.com/list,000001,99,f.html  
网址规律：https://guba.eastmoney.com/list,+code+,99,f_{i}.html

**代码结构：**  
conceptual layer: download_comments(codes)  # 用于聚焦某个个股页面，在多个个股层面操作，output为所有个股合并的长表
logical_layer: parse_pages(url, code)  # 用于聚焦单个个股网页的信息提取，在该个股不同页之间操作，output 为该个股的资讯信息合并长表
physical_layer: get_max_page(url), download_page(url, code)  # 用于设置提取的最大页数，避免资源过度消耗；是改进功能之一，暂时不用。

In [166]:
def download_comments(codes):
concat_container = []
for code in codes:  # 循环遍历数据项
print(code)
url = "https://guba.eastmoney.com/list," + str(code) + ",99,f.html"
concat_container.append(parse_pages(url, code)) # 将每个个股的资讯拼接在一起  # 向列表添加数据
time.sleep(random.uniform(60, 70)) # 通过二分法测试，判断出 1 分钟左右间歇时间可行。当然，可以继续优化效率。  # 循环遍历数据项
concat_df = pd.concat(concat_container, ignore_index = True)
return concat_df

In [167]:
# 解析每个页面的数据
def parse_pages(url, code):
max_page = 5 # 最多截取 50 页的信息，用的时候可以更改； 此处用 5 页，获取最近半年到一年的散户评价；
concat_container = []
for i in range(1, max_page + 1):  # 循环遍历数据项
if i != 1:  # 条件判断
url = "https://guba.eastmoney.com/list," + str(code) + ",99,f_" + str(i) + ".html"
concat_container.append(download_page(url, code))  # 向列表添加数据
# time.sleep(random.uniform(1, 3))
concat_df = pd.concat(concat_container, ignore_index = True) # 将单个个股的每页资讯拼接在一起
return concat_df

In [ ]:
###############待改进功能：最大页数提取【减少资源消耗】
# # 获取url下的最大page数
# def get_max_page(url):
#     page_data = requests.get(url).content.decode("utf-8")
#     data_tree = etree.HTML(page_data)
#     if page_data.find("page_newslib"):
#         max_page = data_tree.xpath("//*[@class=\"page_newslib\"]//a[last()-1]/text()")
#         return int(max_page[0])
#     else:
#         return 1
###############该功能需要用到：ChromeDriver【动态提取页面信息】
# # 设置Selenium浏览器驱动（这里以Chrome为例）
# driver = webdriver.Chrome(executable_path='/Users/wujiewang/Downloads/chrome-mac-arm64/chromedriver')
# # 目标网页URL
# url = 'https://guba.eastmoney.com/list,002422,1,f.html'
# # 使用Selenium打开网页
# driver.get(url)
# # 等待一定时间，让页面加载完成，可能需要根据网速和网页的复杂度调整等待时间
# time.sleep(5)  # 等待5秒
# # 获取页面源代码
# page_data = driver.page_source
# # 解析页面源代码
# data_tree = etree.HTML(page_data)
# # 使用XPath表达式提取所需信息
# element = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/ul/li[7]/a/text()')
# # 输出提取的内容
# print(element)
# # 关闭浏览器
# driver.quit()
###############对网页信息提取进行测试
# url = 'https://guba.eastmoney.com/list,002422,1,f.html'
# page_data = requests.get(url, headers=headers).content.decode("utf-8")
# data_tree = etree.HTML(page_data)
# title = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')

In [168]:
headers = {'User-Agent':
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

**Xpath 格式 for 对应信息**  
- **comment:** //*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr[1]/td[3]/div/a  
- **time:** //*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr[1]/td[5]/div  
- **user_name:** //*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr[1]/td[4]/div/a   
> 获取方式：Chrome-F12-光标指示点击对应信息模块-找到源代码中对应栏目-鼠标右键：copy Xpath

In [32]:
url = 'https://guba.eastmoney.com/list,000001,99,f.html'
page_data = requests.get(url, headers=headers).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)
comments = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')
time = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[5]/div') # 注意一下，说不定可以改
user_name = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[4]/div/a/text()')
time_list = list(map(lambda x: x.text, time))
stock_code = ['000001' for i in range(len(time))]  # 循环遍历数据项

In [33]:
df_content = {
'stock_code': stock_code,
'time': time_list,
'comments': comments,
'user_name': user_name
}
pd.DataFrame(df_content)  # 使用pandas创建DataFrame

,stock_code,time,comments,user_name
0,000001,03-29 07:57,平安银行每股派0.719元，股息率6.9%。兴业银行每股派1.040元，股息率6,海兴大吉
1,000001,03-27 08:38,平安银行目前股息率最高,价值投资棒棒棒
2,000001,03-26 07:13,【炜炜道来】聊聊本轮调整时的操作策略和风险点,边风炜说投资
3,000001,03-24 09:08,部分银行年报业绩快报点评,价值投资棒棒棒
4,000001,03-23 07:15,人们纠结的还是分红，前几天由于平安银行的正面示范作用，长沙走得还挺好，昨天，中信,轻松的苗飞文
...,...,...,...,...
75,000001,02-27 07:41,做银行股的人要耐得住寂寞,白忙呼
76,000001,02-27 03:04,离场的机构，跑步进场,实战老江
77,000001,02-27 06:33,今天北上资金应该没有参与10.5的买卖交易。,月亮小小船
78,000001,02-26 11:39,公认的底部区域跌超6%所谓一阳改三观，前方刚在长停最高打到11.24，今天直接进,平和福


验证结果：👇该网页，时间提取不能用 text（）提取

In [15]:
# url = 'https://guba.eastmoney.com/list,000001,99,f.html'
# page_data = requests.get(url, headers=headers).content.decode("utf-8")
# data_tree = etree.HTML(page_data)
# comments = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')
# time = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[5]/div/a/text()') # 此处有改动
# user_name = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[4]/div/a/text()')
# #time_list = list(map(lambda x: x.text, time))
# stock_code = ['000001' for i in range(len(time))]

# df_content = {
#     'stock_code': stock_code,
#     'time': time_list,
#     'comments': comments,
#     'user_name': user_name
# }
# pd.DataFrame(df_content)

In [169]:
# 解析指定页面的数据并保存至本地
def download_page(url, code):
try:  # 异常处理，尝试执行代码块，处理潜在的错误
headers = {'User-Agent':
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}####根据情况改动
page_data = requests.get(url, headers=headers).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)
comments = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')
times = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[5]/div') # 注意一下，说不定可以改
user_name = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[4]/div/a/text()') # 还没测试
time_list = list(map(lambda x: x.text, times))
stock_code = [str(code) for i in range(len(times))]  # 循环遍历数据项
############## 隐去的代码目的：将每个个股爬取的信息单独输出为一个文件
# save_path = '/Users/wujiewang/Desktop/stocks/' ##### 用的时候改为自己的本地文件夹地址
# file_name = save_path + str(code) + ".txt" # 文件保存路径，确保这个文件夹存在
# for title in titles:
#     title = title + "\r\n"
#     with open(file_name, "a") as file:
#         file.write(title)
#         file.flush()
df_content = {
'stock_code': stock_code,
'time': time_list,
'comments': comments,
'user_name': user_name
}
download_sheet = pd.DataFrame(df_content)  # 使用pandas创建DataFrame
return download_sheet
except:  # 异常处理，尝试执行代码块，处理潜在的错误
print("服务器超时")

In [170]:
stock_info = pd.read_csv('/Users/wujiewang/Desktop/QF5214/inf_stock_data.csv')
stock_info

,Unnamed: 0,EXCH_ENG,SEC_NAME,TRADE_CODE,CONCEPT,HOTCONCEPT,LATESTCONCEPT,CHAIN,COMP_NAME,COMP_NAME_ENG,PROVINCE,NATURE1,INDUSTRY_CSRC2023,INDUSTRY_CSRCCODE2023,INDUSTRY_GICS,INDUSTRY_GICSCODE
0,000001.SZ,SZSE,平安银行,1,跨境支付;银;大盘股;上涨点位贡献;全A等权;银行精选;全A精选;陆股通重仓前100;核心资...,跨境支付;银;上涨点位贡献;银行精选;核心资产;基金重仓,中动量,NaN,平安银行股份有限公司,"Ping An Bank Co., Ltd.",广东省,公众企业,金融业,J,金融,40
1,000002.SZ,SZSE,万科A,2,全A负动量;房地产金股;海通证券金股;券商金股综合;银;大盘股;下跌点位贡献;全A等权;房地...,银;房地产精选;核心资产;分拆上市;基金重仓;新型城镇化,全A负动量,PC 长租公寓,万科企业股份有限公司,"China Vanke Co.,Ltd.",广东省,公众企业,房地产业,K,房地产,60
2,000063.SZ,SZSE,中兴通讯,63,英伟达产业链;6G;新经济;自主可控;TMT;信创产业;通信金股;券商金股综合;长城证券金股...,英伟达产业链;6G;自主可控;信创产业;EDA;核心资产;新基建;5G应用;消费电子产业;半...,全A价格回报,"PC 家庭娱乐,PC 在线办公,PC NCP专利,PC 智能网联汽车,PC 物联网,PC 5...",中兴通讯股份有限公司,Zte Corporation,广东省,公众企业,制造业,C,信息技术,45
3,000069.SZ,SZSE,华侨城A,69,央企房地产;央企中盘股;非陆股通重仓;国企综合;央企综合;中盘股;银;全A等权;全A(非陆股...,国企综合;银;业绩爆雷;社保重仓,央企房地产,NaN,深圳华侨城股份有限公司,"Shenzhen Overseas Chinese Town Co., Ltd",广东省,中央国有企业,房地产业,K,房地产,60
4,000100.SZ,SZSE,TCL科技,100,非基金重仓;新经济;TMT;元宇宙;先进制造;大盘股;下跌点位贡献;三新;双循环;全A等权;...,元宇宙;先进制造;双循环;5G应用;消费电子产业;新型显示技术;分拆上市;科技龙头;数字经济...,非基金重仓,"PC 家庭娱乐,PC 智能手表,PC 消费金融,PC 家电",TCL科技集团股份有限公司,Tcl Technology Group Corporation,广东省,公众企业,制造业,C,信息技术,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,688363.SH,SSE,华熙生物,688363,非陆股通重仓;民营企业综合;口腔医疗;胶原蛋白;中盘股;宁组合;双创100;医美;茅;全A等...,宁组合;医美;茅;基金重仓;生物科技,非陆股通重仓,NaN,华熙生物科技股份有限公司,Bloomage Biotechnology Corporation Limited,山东省,民营企业,制造业,C,医疗保健,35
296,688396.SH,SSE,华润微,688396,央企电子;央企中盘股;非陆股通重仓;央企综合;国企综合;中特估;新经济;自主可控;中盘股;汽...,国企综合;中特估;自主可控;汽车芯片;信创产业;元宇宙;MCU芯片;先进制造;双循环;半导体...,央企电子,NaN,华润微电子有限公司,China Resources Microelectronics Limited,江苏省,中央国有企业,制造业,C,信息技术,45
297,688561.SH,SSE,奇安信-U,688561,非陆股通重仓;全A负动量;民营企业综合;中盘股;TMT;信创产业;跨境支付;元宇宙;下跌点位...,跨境支付;信创产业;元宇宙;双循环;5G应用;科技龙头;基金重仓;操作系统;AIPC;网络安全,非陆股通重仓,NaN,奇安信科技集团股份有限公司,Qi An Xin Technology Group Inc.,北京,民营企业,信息传输、软件和信息技术服务业,I,信息技术,45
298,688599.SH,SSE,天合光能,688599,非基金重仓;民营企业综合;新经济;中资国际化;自主可控;中盘股;TMT;能源出海;TOPco...,自主可控;能源出海;TOPcon电池;先进制造;碳中和;电源设备精选;分拆上市;出海;特高压...,非基金重仓,PC 光伏,天合光能股份有限公司,"Trina Solar Co., Ltd.",江苏省,民营企业,制造业,C,信息技术,45


In [171]:
hold_stocks = list(stock_info.iloc[:, 0].apply(lambda x: x[:6]))
hold_stocks[:5]

['000001', '000002', '000063', '000069', '000100']

In [158]:
##### try a small sample
# 进行正式提取
result6 = pd.DataFrame()  # 使用pandas创建DataFrame
container = []
for i in range(0, 3):  # 循环遍历数据项
result6_i = download_comments(hold_stocks[i:i+1])
if i == 0:  # 条件判断
result6 = result5_i
else:  # 条件判断
container = [result6, result6_i]
result6 = pd.concat(container, ignore_index = True)

000001
000002
000063


In [159]:
result6

,stock_code,time,comments,user_name
0,000063,03-30 02:09,上调市场评级,深耕等待
1,000063,03-29 10:28,美国这个法案我感觉有猫腻不对劲啊大概率是故意做空药明股市的。为什么这么说呢因为这,左侧卧龙在东方
2,000063,03-30 11:29,股票只要在涨，卖了又追高买，其实也不可怕。当年长虹就是很多人买了赚了好几块卖了，,股友5828Ew9782
3,000063,03-29 04:04,纠结的中兴通讯,我心飞翔勇往直前
4,000063,03-28 07:47,华阳:致焦虑的中兴股友3-28,执行者华阳
...,...,...,...,...
1195,000063,11-23 12:02,纵观A股市场来讲的话，如果真的有科技含量的公司来讲，哪个公司的股价会低于50块钱,水中鱼杯中水
1196,000063,11-23 03:16,多年炒股,人类高质量女股民
1197,000063,11-22 04:34,今天收盘，中兴股价26.57元，浪潮36.42元，两者相差近10元；但是今年前三,股友578ai12302
1198,000063,11-22 12:30,大盘三千点以上，中兴通讯下跌空间有限！,耐心持有（股票）


In [160]:
# 进行正式提取
result5 = pd.DataFrame()  # 使用pandas创建DataFrame
container = []
for i in range(0, 300, 50):  # 循环遍历数据项
result5_i = download_comments(hold_stocks[i:i+50])
if i == 0:  # 条件判断
result5 = result5_i
else:  # 条件判断
container = [result5, result5_i]
result5 = pd.concat(container, ignore_index = True)

000001
000002
000063
000069
000100
000157
000166
000301
000333
000338
000408
000425
000538
000568
000596
000617
000625
000651
000661
000708
000725
000733
000768
000776
000786
000792
000800
000858
000876
000877
000895
000938
000963
000977
000983
000999
001289
001979
002001
002007
002027
002049
002050
002074
002129
002142
002179
002180
002202
002230
002236
002241
002252
002271
002304
002311
002352
002371
002410
002415
002459
002460
002466
002475
002493
002555
002594
002601
002603
002648
002709
002714
002736
002812
002821
002841
002916
002920
002938
003816
300014
300015
300033
300059
300122
300124
300142
300223
300274
300308
300316
300347
300408
300413
300433
300450
300454
300496
300498
300628
300661
300750
300751
300759
300760
300763
300782
300896
300919
300957
300979
300999
301269
600000
600009
600010
600011
600015
600016
600018
600019
600023
600025
600028
600029
600030
600031
600036
600039
600048
600050
600061
600085
600089
600104
600111
600115
600132
600150
600176
600183
600188
600196

KeyboardInterrupt: 

出现问题：爬取过程中出现断连，重新爬去未完成部分

In [163]:
result5.to_csv('/Users/wujiewang/Desktop/QF5214/stock_comments_till250.csv')  # 将DataFrame保存到CSV文件，方便后续处理

In [172]:
# 进行正式提取:250-300
result7 = pd.DataFrame()  # 使用pandas创建DataFrame
container = []
for i in range(250, 300):  # 循环遍历数据项
result7_i = download_comments(hold_stocks[i:i+1])
if i == 0:  # 条件判断
result7 = result7_i
else:  # 条件判断
container = [result7, result7_i]
result7 = pd.concat(container, ignore_index = True)

601878
601881
601888
601898
601899
601901
601916
601919
601939
601985
601988
601989
601995
601998
603019
603195
603259
603260
603288
603290
603369
603392
603486
603501
603659
603799
603806
603833
603899
603986
603993
605117
605499
688008
688012
688036
688041
688065
688111
688126
688187
688223
688256
688271
服务器超时
服务器超时


KeyboardInterrupt: 

出现问题：爬取过程中出现断连，重新爬去未完成部分

In [173]:
result7

,stock_code,time,comments,user_name
0,601878,03-30 08:59,浙商证券并未说明实控人已变更,自然自在6003
1,601878,03-30 07:30,其实唱空的人，心里比满仓的人更笃定周一大概率一字涨停,一两酒
2,601878,03-30 12:52,浙商证券的问题浙商证券的问题在于周五（昨天）上午跌3个多点时我清仓了。这股我春节,独立的叶帆7
3,601878,03-30 03:02,全国供销总社收购供销大集 浙商证券,南瑹衡
4,601878,03-30 09:05,最聪明的散户,世昌和丽
...,...,...,...,...
17195,688256,03-04 03:24,没有地板问题不大，继续观望持有，目前还有几个点盈利[哈欠],锅底小卧牛
17196,688256,03-04 03:21,下7.7就重仓，，你跌吧，，，,混迹人间的帝萌
17197,688256,03-04 03:20,快买买买买,好学的华婷1
17198,688256,03-04 02:35,网传蚂蚁把大象吃了，公司也不辟谣！大象有49%是谁的？[坏笑],爱炒股的田鼠


出现问题：爬取过程中出现断连，重新爬去未完成部分

In [175]:
# 进行正式提取:293-300
result8 = pd.DataFrame()  # 使用pandas创建DataFrame
container = []
for i in range(293, 300):  # 循环遍历数据项
result8_i = download_comments(hold_stocks[i:i+1])
if i == 0:  # 条件判断
result8 = result8_i
else:  # 条件判断
container = [result8, result8_i]
result8 = pd.concat(container, ignore_index = True)

688271
688303
688363
688396
688561
688599
688981


In [176]:
result8

,stock_code,time,comments,user_name
0,688271,03-19 04:04,（四）提升教育文旅医疗设备水平。推动符合条件的高校、职业院校（含技工院校）更新置,股友561792Iz72
1,688271,03-08 04:49,牛市大势所趋，民心所向。政策的呵护已成定局，不容置疑！顺势而为，识时务者为俊杰，通机变者为英...,中国悟空
2,688271,02-29 12:47,动态点评：业绩基本符合预期，全球市场大有可为,芯药酒合伙人
3,688271,12-24 10:15,联影医疗是国内医学影像领域的绝对龙头，承担着国内CT、核磁共振、分子影像和X射线,股海有花_小号
4,688271,12-20 07:04,大仙擅长趋势股的把握，全网第一次解密21日均线战法的威力。。。为什么大仙要以,大罗金仙在都市
...,...,...,...,...
2632,688981,08-10 05:53,中芯国际：第二季度净利润4.028亿美元 去年同期5.14亿美元08-10 1,股友SYeROy
2633,688981,08-10 02:21,中芯国际走势跟踪,生动的蔡三仟
2634,688981,08-09 10:18,但使龙城飞将在，不教胡马度阴山！,又想一刀
2635,688981,08-09 04:45,九成中芯散户在看做市商的笑话,梦回六朝


In [177]:
result8.groupby(by = 'stock_code').count()

,time,comments,user_name
stock_code,,,
688271,400,400,400
688303,400,400,400
688363,400,400,400
688396,399,399,399
688561,238,238,238
688599,400,400,400
688981,400,400,400


In [179]:
for i in ['688396', '688561']:  # 循环遍历数据项
print(i)

688396
688561


In [180]:
# 进行正式提取:293-300
result9 = pd.DataFrame()  # 使用pandas创建DataFrame
container = []
temp_list = ['688396', '688561']
for i in range(2):  # 循环遍历数据项
result9_i = download_comments(temp_list[i:i+1])
if i == 0:  # 条件判断
result9 = result9_i
else:  # 条件判断
container = [result9, result9_i]
result9 = pd.concat(container, ignore_index = True)

688396
688561


In [181]:
result9

,stock_code,time,comments,user_name
0,688396,03-27 04:23,华润微和长电科技有哪些业务可以协同发展。,股海是江湖
1,688396,03-18 11:12,证监会15号发文严厉打击“伪市值管理”，试问华润微公司内在有没有内幕交易，7大坐,基民0Tz8u8
2,688396,03-07 05:44,华润微：SiC目前产能达到2500片月,集邦化合物半导体
3,688396,02-25 07:06,说实话，最近想了一下，这股所谓的重产品路线都是功率半导体器件，以三代为主，主要供,股友72g273o927
4,688396,02-23 03:33,华润微：目前8吋产能利用率保持在90%以上 6吋保持在95%以上,芯药酒合伙人
...,...,...,...,...
632,688561,07-23 10:13,机构博弈的时代开启了,AAAAA2017
633,688561,07-22 01:14,前天止盈启明星辰，持有了138天，赚了19%。后来深入研究发现奇安信技术实力最强,我是金先生
634,688561,07-14 10:57,科创50的股票，上市满一年后的巨额解禁就没有不跌的，而且下跌个10%起步，现在还,neal0526
635,688561,07-13 03:44,洗了一天盘子差不多了吧，下午成交量只有上午的20%，本韭菜今天119入场来奇安信,坐看云起2501


In [182]:
result_all = pd.concat([result5, result7, result8], ignore_index = True)

In [186]:
len(set(result_all['stock_code']))

300

In [189]:
result_all.to_csv('/Users/wujiewang/Desktop/QF5214/stock_comments.csv', encoding='utf_8_sig', index=False) # 输出结果  # 将DataFrame保存到CSV文件，方便后续处理
result_all.to_csv('/Users/wujiewang/Desktop/QF5214/stock_comments')  # 将DataFrame保存到CSV文件，方便后续处理

## **测试功能：**加入帖子内容栏，提取出散户股民的具体评论内容（未完）
目前问题在于：重复访问帖子页面极易导致锁 IP；设置 time.sleep 爬取速度过慢。

In [60]:
url = 'https://guba.eastmoney.com/list,000001,99,f.html'
page_data = requests.get(url, headers=headers).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)
comments_raw_link = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/@href')

In [98]:
comments_link = ['https:' + raw_link for raw_link in comments_raw_link]  # 循环遍历数据项
comments_link[:7]

['https://caifuhao.eastmoney.com/news/20240329075715153732760',
 'https://caifuhao.eastmoney.com/news/20240327203804395624330',
 'https://caifuhao.eastmoney.com/news/20240326191342702518150',
 'https://caifuhao.eastmoney.com/news/20240324210807406631390',
 'https://caifuhao.eastmoney.com/news/20240323071540290319170',
 'https://caifuhao.eastmoney.com/news/20240323152137165815210',
 'https://caifuhao.eastmoney.com/news/20240321203157983806240']

In [97]:
##### 测试第一个网页
url ='https://caifuhao.eastmoney.com/news/20240323071540290319170'  # 条件判断
page_data = requests.get(url, headers=headers).content.decode("utf-8")# 进入帖子，获取源代码  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data) # 格式规范化网页源代码
comments_raw_content1 = data_tree.xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/p/text()')# 提取相关信息,格式 1
comments_raw_content2 = comments_raw_content = data_tree.xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/p/span/text()')# 提取相关信息，格式 2
comments_raw_content = comments_raw_content1 + comments_raw_content2
comments_content = ''.join(comments_raw_content)

In [95]:
comments_content

'\xa0\xa0人们纠结的还是分红，前几天由于平安银行的正面示范作用，长沙走得还挺好，昨天，中信银行的反面示范作用又将它带偏了。现在监管部门对分红没有硬性规定，只是提倡啊，鼓励啊，没有用，最后还得看公司的良心。如果长沙银行分红能向平安行一样，也来个30%，那么，它的股价马上就会有起色，将会比江行高出一截。看了董秘对投资者的回复，好象挺难的。大家翘首以待吧，不过，再跌一点，还是有价值的。\xa0'

In [85]:
#### 测试第二个网页
url ='https://caifuhao.eastmoney.com/news/20240324210807406631390'  # 条件判断
page_data = requests.get(url, headers=headers).content.decode("utf-8")# 进入帖子，获取源代码  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data) # 格式规范化网页源代码
comments_raw_content = data_tree.xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/p/span/text()')# 提取相关信息
comments_content = ''.join(comments_raw_content)

In [86]:
comments_content

'\xa0杭州银行2023年，杭州银行全年实现营业收入350.16亿元，较上年同期增长6.33%；实现归属于上市公司股东的净利润143.83亿元，较上年同期增长23.15%。截至2023年末，公司总资产18413.42亿元，较上年末增长13.91%；贷款总额8070.96亿元，较上年末增长14.94%；公司不良贷款率0.76%，较上年末下降0.01个百分点；拨备覆盖率561.42%，较上年末下降3.68个百分点 。目前市值642亿元。目前动态市盈率4.5倍。2023年报大概率每股红利0.45元。缺点：分红在银行股中中等，核心资本充足率也较低，仅8.31%，有再融资需求，定向增发预案已发，只不过因为破净破发和市场低迷而搁置。不过核心资本充足率已经连续三个季度增加了，由8.08%—8.1%—8.14%—8.31%。看样子有靠内生性增长也能补充核心资本。有150亿可转债，目前基本没有转股。招商银行：1月19日晚间发布业绩快报，2023年实现营业收入3391.23亿元，同比下降1.64%；净利润1466.02亿元，同比增长6.22%；基本每股收益5.63元。截至2023年三季度，不良贷款率0.96%，较上年末持平；拨备覆盖率445.86%，较上年末下降4.93个百分点；贷款拨备率4.28%，较上年末下降0.04个百分点。截至2023年12月31日，不良贷款率0.95%，较上年末下降0.01个百分点；拨备覆盖率437.70%，较上年末下降13.09个百分点；贷款拨备率4.14%，较上年末下降0.18个百分点。招商银行降低拨备率保障业绩，资产质量在上市银行占中上等，低于部分优秀的城商行和农商行。长沙银行发布业绩快报。2023年，公司实现营业收入248.03亿元，同比增长8.46%；归属于上市公司股东的净利润74.63亿元，同比增长9.57%。2023年末，不良贷款率1.15%，较年初下降0.01个百分点；拨备覆盖率314.21%，较年初上升3.12个百分点。长沙银行：不到4.4倍市盈率。中信银行：发布年报。2023年公司实现营业收入约2059亿元，同比降2.6%；归属于上市公司股东的净利润约670.16亿元，同比增长7.91%；基本每股收益1.14元。不良贷款实现双降 。截至2023年末，不良贷款率1.18%，较年初下降0.09个百分点；拨备覆盖率208.58%，较年初提高6.

In [87]:
#### 测试第三个网页
url ='https://caifuhao.eastmoney.com/news/20240329075715153732760'  # 条件判断
page_data = requests.get(url, headers=headers).content.decode("utf-8")# 进入帖子，获取源代码  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data) # 格式规范化网页源代码
comments_raw_content = data_tree.xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/p/text()')# 提取相关信息
comments_content = ''.join(comments_raw_content)

In [88]:
comments_content

'平安银行每股派0.719元，股息率6.9%。兴业银行每股派1.040元，股息率6.4%。招商银行每股派1.972元，股息率6.1%。交通银行每股派0.375元，股息率6.0%。中信银行每股派0.356元，股息率5.8%。工商银行每股派0.306元，股息率5.8%。建设银行每股派0.400元，股息率5.8%。农业银行每股派0.231元，股息率5.5%。邮储银行每股派0.261元，股息率5.5%。中国银行每股派0.236元，股息率5.4%。民生银行每股派0.216元，股息率5.3%。'

In [99]:
#### 测试第四个网页
url ='https://caifuhao.eastmoney.com/news/20240323152137165815210'  # 条件判断
page_data = requests.get(url, headers=headers).content.decode("utf-8")# 进入帖子，获取源代码  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data) # 格式规范化网页源代码
comments_raw_content1 = data_tree.xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/p/text()')# 提取相关信息,格式 1
comments_raw_content2 = comments_raw_content = data_tree.xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/p/span/text()')# 提取相关信息，格式 2
comments_raw_content = comments_raw_content1 + comments_raw_content2
comments_content = ''.join(comments_raw_content)

In [100]:
comments_content

'\xa0 \xa0 \xa0 最近看了很多很多的短视频老师的直播，基本可以囊括为卖课程和指标。也不能说是有点东西，基本是毫无任何作用。\xa0 \xa0 \xa0 从业15年来，经历过很多风风雨雨，我想在这个市场想要赚到钱，就必须构建自己的交易系统。很多股民我只能用两个字概括：头铁。明明是一直亏钱，偏偏就不愿意去寻找学习新的方法，总觉得自己的是对的，别人的都不可靠。其实判断你所用的交易模式是否正确，只有一个标准：能否能稳定的盈利。这是这个市场唯一的检测标准。\xa0 \xa0 \xa0 要找到这个方法，你只有不断的去学习，实践，统计。\xa0 \xa0 \xa0 举个简单的例子： 这是平安银行的日线图，我们先做一个简单的交易系统：跟着MACD的金叉死叉买卖。从2023.6.8的金叉开始，我们做详细的数据统计：6.8收盘价11.84我们买入，6.26死叉离场，收盘价11.18（亏损），7.3再次建仓，收盘价11.49，8.14离场，收盘价11.67（有点利润）。虽然后面9.6-10.10以震荡为主，但是后续行情我们是无法预测的，当你打算构建这个交易西永就必须严谨，不能有侥幸心理。所以9.6会再次开仓，9.20亏损立场，9.25开仓，10.10亏损立场。11.6开仓，11.17亏损立场。12.22开仓，2024.3.6立场（有7毛钱利润）。为期几乎大半年，最终结果是亏损立场。所以这个交易系统可以直接pass掉。\xa0 \xa0 \xa0 \xa0当然这里只是简单的做一个指引而已，构建交易体系，可以参照的东西很多很多。当然你每用一个就必须去做大量的数据统计和轻仓的实战统计。今天先简单说到这里，祝大家龙年顺风顺水，财运滚滚。\xa0 \xa0 \xa0'

**上述结果表明，格式 1 与格式 2 测试成功。**

In [122]:
#### 获得单个帖子的内容
def fetch_content(input_url, headers):
url = input_url
page_data = requests.get(url, headers=headers).content.decode("utf-8")# 进入帖子，获取源代码  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data) # 格式规范化网页源代码
comments_raw_content1 = data_tree.xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/p/text()')# 提取相关信息,格式 1
comments_raw_content2 = comments_raw_content = data_tree.xpath('//*[@id="main"]/div[2]/div[1]/div[1]/div[1]/div[3]/div[1]/p/span/text()')# 提取相关信息，格式 2
comments_raw_content = comments_raw_content1 + comments_raw_content2
comments_content = ''.join(comments_raw_content)
return comments_content

In [138]:
fetch_content('https://caifuhao.eastmoney.com/news/20240323152137165815210', headers)  # 条件判断

'\xa0 \xa0 \xa0 最近看了很多很多的短视频老师的直播，基本可以囊括为卖课程和指标。也不能说是有点东西，基本是毫无任何作用。\xa0 \xa0 \xa0 从业15年来，经历过很多风风雨雨，我想在这个市场想要赚到钱，就必须构建自己的交易系统。很多股民我只能用两个字概括：头铁。明明是一直亏钱，偏偏就不愿意去寻找学习新的方法，总觉得自己的是对的，别人的都不可靠。其实判断你所用的交易模式是否正确，只有一个标准：能否能稳定的盈利。这是这个市场唯一的检测标准。\xa0 \xa0 \xa0 要找到这个方法，你只有不断的去学习，实践，统计。\xa0 \xa0 \xa0 举个简单的例子： 这是平安银行的日线图，我们先做一个简单的交易系统：跟着MACD的金叉死叉买卖。从2023.6.8的金叉开始，我们做详细的数据统计：6.8收盘价11.84我们买入，6.26死叉离场，收盘价11.18（亏损），7.3再次建仓，收盘价11.49，8.14离场，收盘价11.67（有点利润）。虽然后面9.6-10.10以震荡为主，但是后续行情我们是无法预测的，当你打算构建这个交易西永就必须严谨，不能有侥幸心理。所以9.6会再次开仓，9.20亏损立场，9.25开仓，10.10亏损立场。11.6开仓，11.17亏损立场。12.22开仓，2024.3.6立场（有7毛钱利润）。为期几乎大半年，最终结果是亏损立场。所以这个交易系统可以直接pass掉。\xa0 \xa0 \xa0 \xa0当然这里只是简单的做一个指引而已，构建交易体系，可以参照的东西很多很多。当然你每用一个就必须去做大量的数据统计和轻仓的实战统计。今天先简单说到这里，祝大家龙年顺风顺水，财运滚滚。\xa0 \xa0 \xa0'

In [131]:
# 解析指定页面的数据并保存至本地
def download_page_w_content(url, code):
try:  # 异常处理，尝试执行代码块，处理潜在的错误
headers = {'User-Agent':
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}####根据情况改动
page_data = requests.get(url, headers=headers).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)
comments = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')
times = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[5]/div') # 注意一下，说不定可以改
user_name = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[4]/div/a/text()') # 还没测试
time_list = list(map(lambda x: x.text, times))
stock_code = [str(code) for i in range(len(times))]  # 循环遍历数据项
#### 提取帖子内容
comments_raw_link = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/@href')
comments_link = ['https:' + raw_link for raw_link in comments_raw_link]  # 循环遍历数据项
comments = []
for link in comments_link:  # 循环遍历数据项
comments.append(fetch_content(link, headers)) # 提取帖子内的内容  # 向列表添加数据
time.sleep(random(7, 10))
############## 隐去的代码目的：将每个个股爬取的信息单独输出为一个文件
# save_path = '/Users/wujiewang/Desktop/stocks/' ##### 用的时候改为自己的本地文件夹地址
# file_name = save_path + str(code) + ".txt" # 文件保存路径，确保这个文件夹存在
# for title in titles:
#     title = title + "\r\n"
#     with open(file_name, "a") as file:
#         file.write(title)
#         file.flush()
df_content = {
'stock_code': stock_code,
'time': time_list,
'comment_title': comments,
'user_name': user_name,
'comment_content': comments
}
download_sheet = pd.DataFrame(df_content)  # 使用pandas创建DataFrame
return download_sheet
except:  # 异常处理，尝试执行代码块，处理潜在的错误
print("服务器超时")

In [127]:
def download_comments2(codes):
concat_container = []
for code in codes:  # 循环遍历数据项
print(code)
url = "https://guba.eastmoney.com/list," + str(code) + ",99,f.html"
concat_container.append(parse_pages2(url, code)) # 将每个个股的资讯拼接在一起  # 向列表添加数据
time.sleep(random.uniform(30, 40)) # 通过二分法测试，判断出 1 分钟左右间歇时间可行。当然，可以继续优化效率。  # 循环遍历数据项
concat_df = pd.concat(concat_container, ignore_index = True)
return concat_df

In [128]:
# 解析每个页面的数据
def parse_pages2(url, code):
max_page = 5 # 最多截取 50 页的信息，用的时候可以更改； 此处用 5 页，获取最近半年到一年的散户评价；
concat_container = []
for i in range(1, max_page + 1):  # 循环遍历数据项
if i != 1:  # 条件判断
url = "https://guba.eastmoney.com/list," + str(code) + ",99,f_" + str(i) + ".html"
concat_container.append(download_page_w_content(url, code))  # 向列表添加数据
# time.sleep(random.uniform(1, 3))
concat_df = pd.concat(concat_container, ignore_index = True) # 将单个个股的每页资讯拼接在一起
return concat_df

###############可能存在问题： 访问频率过快，导致封禁；访问频率过慢，效率过低；

In [ ]:
####下一个问题：如何做到爬一个个股存一个数据？ 按个股来？

In [134]:
download_page_w_content('https://guba.eastmoney.com/list,000001,99,f.html', '000001')

服务器超时


In [132]:
##### try a small sample
# 进行正式提取
result5 = pd.DataFrame()  # 使用pandas创建DataFrame
container = []
for i in range(0, 3):  # 循环遍历数据项
result5_i = download_comments2(hold_stocks[i:i+1])
if i == 0:  # 条件判断
result5 = result5_i
else:  # 条件判断
container = [result5, result5_i]
result5 = pd.concat(container, ignore_index = True)

000001
服务器超时
服务器超时
服务器超时
服务器超时
服务器超时


ValueError: All objects passed were None

In [ ]:
可能存在问题：
1、访问频率过快，导致服务器脱节
2、二者不能同时进行，需要提取。------如果可以，那就分开成两个板块信息，确定好拼接索引列即可。

In [111]:
result5

,stock_code,time,comments,user_name
0,000001,03-29 07:57,平安银行每股派0.719元，股息率6.9%。兴业银行每股派1.040元，股息率6,海兴大吉
1,000001,03-27 08:38,平安银行目前股息率最高,价值投资棒棒棒
2,000001,03-26 07:13,【炜炜道来】聊聊本轮调整时的操作策略和风险点,边风炜说投资
3,000001,03-24 09:08,部分银行年报业绩快报点评,价值投资棒棒棒
4,000001,03-23 07:15,人们纠结的还是分红，前几天由于平安银行的正面示范作用，长沙走得还挺好，昨天，中信,轻松的苗飞文
...,...,...,...,...
1195,000063,11-23 12:02,纵观A股市场来讲的话，如果真的有科技含量的公司来讲，哪个公司的股价会低于50块钱,水中鱼杯中水
1196,000063,11-23 03:16,多年炒股,人类高质量女股民
1197,000063,11-22 04:34,今天收盘，中兴股价26.57元，浪潮36.42元，两者相差近10元；但是今年前三,股友578ai12302
1198,000063,11-22 12:30,大盘三千点以上，中兴通讯下跌空间有限！,耐心持有（股票）


In [155]:
url = 'https://guba.eastmoney.com/list,000001,99,f.html'
headers = {'User-Agent':
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}####根据情况改动
page_data = requests.get(url, headers=headers).content.decode("utf-8")  # 使用requests库发送GET请求获取网页内容
data_tree = etree.HTML(page_data)
comments = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/text()')
times = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[5]/div') # 注意一下，说不定可以改
user_name = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[4]/div/a/text()') # 还没测试
time_list = list(map(lambda x: x.text, times))
stock_code = ['000001' for i in range(len(times))]  # 循环遍历数据项
#### 提取帖子内容
comments_raw_link = data_tree.xpath('//*[@id="mainlist"]/div/ul/li[1]/table/tbody/tr/td[3]/div/a/@href')
comments_link = ['https:' + raw_link for raw_link in comments_raw_link]  # 循环遍历数据项
comments = []
for link in comments_link:  # 循环遍历数据项
try:  # 异常处理，尝试执行代码块，处理潜在的错误
comments.append(fetch_content(link, headers)) # 提取帖子内的内容  # 向列表添加数据
print('succeed')
except:  # 异常处理，尝试执行代码块，处理潜在的错误
comments.append('Failed to fetch content.')  # 向列表添加数据
print('fail')
time.sleep(random.uniform(5, 10)) # 预计3分钟左右检查一次爬虫，因此时间设置长一点。  # 循环遍历数据项

succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
fail
succeed
succeed
fail
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
fail
succeed
succeed
fail
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
fail
fail
succeed
succeed
fail
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
succeed
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail
fail


In [ ]:
# 进行正式提取
result5 = pd.DataFrame()  # 使用pandas创建DataFrame
container = []
for i in range(0, 300, 50):  # 循环遍历数据项
result5_i = download_news(hold_stocks[i:i+50])
if i == 0:  # 条件判断
result5 = result5_i
else:  # 条件判断
container = [result5, result5_i]
result5 = pd.concat(container, ignore_index = True)

In [ ]:
result5